In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# Homework description

Implement simple mixture of experts:

- ROUTER: clusterize your data (on features). Create 2 clusters.
- EXPERTS: Build experts (GB ensamble M=10) per clusters
- AGGREGATOR: you should define it. You can choose: Argmax (probabilities), wieghted sum, NN over expert's outputs and probabilities

In [77]:
train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [78]:
numeric_columns = [i for i, j in zip(train_data.columns, train_data.dtypes) if j in [np.int64, np.float64] and i not in ['SalePrice', 'Id']]
x_train = train_data[numeric_columns].fillna(-1)
x_test = test_data[numeric_columns].fillna(-1)

In [79]:
y_train = np.log(train_data['SalePrice'])

In [80]:
def rmse(a, b):
    return ((a - b) ** 2).mean() ** 0.5

In [81]:
from sklearn.cluster import KMeans

N_CLUSTERS = 2
KMEANS_RANDOM_STATE = 0

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=KMEANS_RANDOM_STATE).fit(x_train)
np.unique(kmeans.labels_, return_counts=True)

(array([0, 1], dtype=int32), array([   4, 1456]))

In [82]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler().fit(x_train)
x_train = standard_scaler.transform(x_train)
x_test = standard_scaler.transform(x_test)

In [83]:
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=KMEANS_RANDOM_STATE).fit(x_train)
np.unique(kmeans.labels_, return_counts=True)

(array([0, 1], dtype=int32), array([714, 746]))

In [84]:
x_train = pd.DataFrame(x_train, columns=numeric_columns)
x_test = pd.DataFrame(x_test, columns=numeric_columns)
x_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.073375,0.216151,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514125,0.575425,-0.288653,...,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
1,-0.872563,0.645357,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570710,1.171992,-0.288653,...,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
2,0.073375,0.301992,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325940,0.092907,-0.288653,...,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
3,0.309859,0.073083,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570710,-0.499274,-0.288653,...,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
4,0.073375,0.759812,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366496,0.463568,-0.288653,...,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.073375,0.130310,-0.260560,-0.071836,-0.517200,0.918511,0.733308,-0.570710,-0.973018,-0.288653,...,-0.060731,-0.752176,-0.100558,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.620891,-0.614439
1456,-0.872563,0.788426,0.266407,-0.071836,0.381743,0.222975,0.151865,0.087940,0.759659,0.722112,...,0.126420,2.033231,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,1.645210
1457,0.309859,0.244765,-0.147810,0.651479,3.078570,-1.002492,1.024029,-0.570710,-0.369871,-0.288653,...,-1.033914,-0.752176,0.201405,-0.359325,-0.116339,-0.270208,-0.068692,4.953112,-0.489110,1.645210
1458,-0.872563,0.301992,-0.080160,-0.795151,0.381743,-0.704406,0.539493,-0.570710,-0.865548,6.092188,...,-1.090059,2.168910,-0.704483,1.473789,-0.116339,-0.270208,-0.068692,-0.087688,-0.859110,1.645210


In [85]:
x_train['Cluster'] = kmeans.labels_
x_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Cluster
0,0.073375,0.216151,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514125,0.575425,-0.288653,...,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777,0
1,-0.872563,0.645357,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570710,1.171992,-0.288653,...,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439,1
2,0.073375,0.301992,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325940,0.092907,-0.288653,...,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777,0
3,0.309859,0.073083,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570710,-0.499274,-0.288653,...,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655,1
4,0.073375,0.759812,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366496,0.463568,-0.288653,...,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.073375,0.130310,-0.260560,-0.071836,-0.517200,0.918511,0.733308,-0.570710,-0.973018,-0.288653,...,-0.752176,-0.100558,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.620891,-0.614439,0
1456,-0.872563,0.788426,0.266407,-0.071836,0.381743,0.222975,0.151865,0.087940,0.759659,0.722112,...,2.033231,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,1.645210,0
1457,0.309859,0.244765,-0.147810,0.651479,3.078570,-1.002492,1.024029,-0.570710,-0.369871,-0.288653,...,-0.752176,0.201405,-0.359325,-0.116339,-0.270208,-0.068692,4.953112,-0.489110,1.645210,0
1458,-0.872563,0.301992,-0.080160,-0.795151,0.381743,-0.704406,0.539493,-0.570710,-0.865548,6.092188,...,2.168910,-0.704483,1.473789,-0.116339,-0.270208,-0.068692,-0.087688,-0.859110,1.645210,1


**Catboost only, avg ensemble**

In [87]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from tqdm import tqdm

M = 10

x_train_clustered_list = [x_train[x_train['Cluster'] == i].drop(columns=['Cluster']) for i in range(N_CLUSTERS)]
y_train_clustered_list = [y_train[x_train[x_train['Cluster'] == i].index] for i in range(N_CLUSTERS)]

experts_rmses_tr = []
experts_rmses_val = []
experts_preds_test = []

for x_train_clustered, y_train_clustered in zip(x_train_clustered_list, y_train_clustered_list):
    rmses_tr = []
    rmses_val = []
    y_preds_test = []
    for m in tqdm(range(M)):
        x_tr, x_val, y_tr, y_val = train_test_split(x_train_clustered, y_train_clustered, test_size=0.1, random_state=m)
        model = CatBoostRegressor(verbose=False).fit(x_tr, y_tr)
        y_pred_tr = model.predict(x_tr)
        rmses_tr.append(rmse(y_tr, y_pred_tr))
        y_pred_val = model.predict(x_val)
        rmses_val.append(rmse(y_val, y_pred_val))
        y_preds_test.append(model.predict(x_test))
    experts_rmses_tr.append(np.exp(np.mean(rmses_tr, axis=0)))
    experts_rmses_val.append(np.exp(np.mean(rmses_val, axis=0)))    
    experts_preds_test.append(np.exp(np.mean(y_preds_test, axis=0)))    

100%|██████████| 10/10 [00:21<00:00,  2.19s/it]


In [88]:
experts_rmses_tr

[1.0235643371732541, 1.030366035946671]

In [89]:
experts_rmses_val

[1.1351729526581429, 1.1533165605504823]

In [57]:
avg_experts_preds_test = np.mean(experts_preds_test, axis=0)
avg_experts_preds_test

array([142979.13265101, 161598.46646019, 179356.06414367, ...,
       191202.62641849, 133283.41434865, 211027.64328954])

In [58]:
submit_avg = pd.DataFrame()
submit_avg['Id'] = test_data['Id']
submit_avg['SalePrice'] = avg_experts_preds_test

submit_avg.to_csv('/kaggle/working/mixture_of_experts_avg.csv', index=False)

**Different GB, wieghted sum ensemble**

In [94]:
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
# !pip install tensorflow_decision_forests
import tensorflow_decision_forests as tfdf

gbs = [
    ('sklearn', GradientBoostingRegressor(), 0.05),
    ('sklearn', GradientBoostingRegressor(), 0.05),
    ('lgb', LGBMRegressor(), 0.1),
    ('lgb', LGBMRegressor(), 0.1),
    ('xgb', XGBRegressor(), 0.1),
    ('xgb', XGBRegressor(), 0.1),
    ('keras', tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.REGRESSION, verbose=0), 0.1),
    ('keras', tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.REGRESSION, verbose=0), 0.1),
    ('catb', CatBoostRegressor(verbose=False), 0.15),
    ('catb', CatBoostRegressor(verbose=False), 0.15),
]

experts_rmses_tr = []
experts_rmses_val = []
experts_preds_test = []

for x_train_clustered, y_train_clustered in zip(x_train_clustered_list, y_train_clustered_list):
    rmses_tr = []
    rmses_val = []
    y_preds_test = []
    for m in tqdm(range(M)):
        x_tr, x_val, y_tr, y_val = train_test_split(x_train_clustered, y_train_clustered, test_size=0.1, random_state=m)
        if gbs[m][0] != 'keras':
            model = gbs[m][1].fit(x_tr, y_tr)
            
            y_pred_tr = model.predict(x_tr)
            y_pred_val = model.predict(x_val)
            y_pred_test = model.predict(x_test)
        else:
            tr = pd.concat([x_tr, y_tr], axis=1)
            tf_tr = tfdf.keras.pd_dataframe_to_tf_dataset(tr, label='SalePrice', task=tfdf.keras.Task.REGRESSION)
            
            model = tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.REGRESSION, verbose=0)
            model.fit(x=tf_tr, verbose=0)

            y_pred_tr = model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(x_tr, task=tfdf.keras.Task.REGRESSION), verbose=0)
            y_pred_val = model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(x_val, task=tfdf.keras.Task.REGRESSION), verbose=0)
            y_pred_test = model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(x_test, task=tfdf.keras.Task.REGRESSION), verbose=0)

            y_pred_tr = np.array([pred_tr_el for pred_tr in y_pred_tr for pred_tr_el in pred_tr])
            y_pred_val = np.array([pred_val_el for pred_val in y_pred_val for pred_val_el in pred_val])
            y_pred_test = np.array([pred_test_el for pred_test in y_pred_test for pred_test_el in pred_test])
                        
        rmses_tr.append(rmse(y_tr, y_pred_tr))
        rmses_val.append(rmse(y_val, y_pred_val))
        y_preds_test.append(gbs[m][2] * y_pred_test)
    
    experts_rmses_tr.append(np.exp(np.mean(rmses_tr, axis=0)))
    experts_rmses_val.append(np.exp(np.mean(rmses_val, axis=0)))    
    experts_preds_test.append(np.exp(np.sum(y_preds_test, axis=0)))    

 60%|██████    | 6/10 [00:01<00:01,  3.00it/s]/opt/conda/lib/python3.7/site-packages/tensorflow_decision_forests/keras/core.py:1612: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
[INFO kernel.cc:736] Start Yggdrasil model training
[INFO kernel.cc:737] Collect training examples
[INFO kernel.cc:392] Number of batches: 11
[INFO kernel.cc:393] Number of examples: 642
[INFO kernel.cc:759] Dataset:
Number of records: 642
Number of columns: 37

Number of columns by type:
	NUMERICAL: 37 (100%)

Columns:

NUMERICAL: 37 (100%)
	0: "1stFlrSF" NUMERICAL mean:0.479986 min:-1.72757 max:9.13268 sd:1.11765
	1: "2ndFlrSF" NUMERICAL mean:0.348767 min:-0.795163 max:3.93696 sd:1.14877
	2: "3SsnPorch" NUMERICAL mean:0.0648964 min:-0.116339 max:17.2172 sd:1.31002
	3: "BedroomAbvGr" NUMERICAL mean:0.21726 min:-3.51495 max:6.295 sd:0.997449
	4: "BsmtFinSF1" NUMERICAL mean:0.1770

In [95]:
experts_rmses_tr

[1.0455093569627287, 1.0427789119480244]

In [96]:
experts_rmses_val

[1.1496983379286516, 1.1614916488904745]

In [97]:
avg_experts_preds_test = np.mean(experts_preds_test, axis=0)
avg_experts_preds_test

array([142523.91729965, 163415.35183128, 179599.84978026, ...,
       188298.11941758, 136007.41864971, 205206.74046762])

In [98]:
submit_weighted_ensemble = pd.DataFrame()
submit_weighted_ensemble['Id'] = test_data['Id']
submit_weighted_ensemble['SalePrice'] = avg_experts_preds_test

submit_weighted_ensemble.to_csv('/kaggle/working/mixture_of_experts_weighted_ensemble.csv', index=False)